In [315]:
import os
import pandas as pd
import re
import numpy as np

In [395]:
def loadGFA():
    filename = os.path.join("../Inputs/GFADataset.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} not found")
    df_GFA = pd.read_csv(filename, sep=",")
    dfGFA=df_GFA.rename(columns={"Alloy Formula":"Formula","Phase Formation":"Phase"})
    return dfGFA
    #print(dfGFA)

def loadTablaPeriodica():
    filename = os.path.join("../Inputs/TablaPeriodica.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} not found")
    df_Tabla = pd.read_csv(filename, sep=";")
    df_TablaPeriodica=df_Tabla.rename(columns={"Eea (ev)":"Eea(ev)","I1 (ev)":"I1(ev)","I2 (ev)":"I2(ev)","Tm (K)":"Tm(K)","Rm (nm)":"Rm(nm)","Rc (nm)":"Rc(nm)","Cp (J/molK)":"Cp(J/molK)","K (W/m)/K 300K":"K(W/m)/K300K","Hf (kJ/mol)":"Hf(kJ/mol)","Tb (K)":"Tb(K)"})
    return df_TablaPeriodica
    #print(df_Tabla)

def loadErroresModelo():
    module_path = os.path.dirname(__file__)
    filename = os.path.join(module_path, "../Inputs/ErroresModelo.csv")
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} not found")
    df_ErroresModelo = pd.read_csv(filename, sep=";")
    return df_ErroresModelo

In [394]:
def formulaWithoutNumber(dfGFA):
    #Definir un df para guardar el resulrado
    dfCompound = pd.DataFrame(columns =['Formula', 'Phase'])
    #Para cada copuesto en el archivo que solo tenga caracteres, se hace la separacion por elementos
    # y se divide 100 entre la cantidad de elementos encontrados
    for index,row in dfGFA.iterrows():
        formulaTx = row['Formula']
        Phase = row['Phase']
        #Seleccionar compuestos sin valores numericos
        if str.isalpha(formulaTx):
            for char in formulaTx:
                if char.isupper():
                    element = re.findall('[A-Z][^A-Z]*', formulaTx)
            cantComp=str(round(100/(len(element)),1))
            lisComponentsCantComponent = [x + cantComp for x in element]
            strComponentsCantComponent = ''.join([str(elem) for elem in lisComponentsCantComponent])
            dfCompound.loc[len(dfCompound)] = strComponentsCantComponent,Phase       
    return(dfCompound)

In [393]:
def findCompositionPercentage(example_compuesto):
    if example_compuesto.count('(') > 0:
        #Si el compuesto tiene parentesis al inicio y al final, pasar
        if example_compuesto.startswith('(') and example_compuesto.endswith(')'):
            #Eliminar los parentesis
            example_compuesto = example_compuesto.replace('(', '')
            example_compuesto = example_compuesto.replace(')', '')
        else:
            match = re.search(r'\((.*?)\)(\d+\.?\d*)', example_compuesto)
            compuesto = match.group(1)
            multiplicador = match.group(2)
            compuesto = re.sub(r'(\d+\.?\d*)', lambda x: str(float(x.group(1))* float(multiplicador)), compuesto)

            #Reemplazar el compuesto en la formula
            example_compuesto = example_compuesto.replace(match.group(0), compuesto)

    if example_compuesto.count('[') > 0:
        match = re.search(r'\[(.*?)\](\d+\.?\d*)', example_compuesto)
        compuesto = match.group(1)
        multiplicador = match.group(2)

        compuesto = re.sub(r'(\d+\.?\d*)', lambda x: str(float(x.group(1))* float(multiplicador)), compuesto)

        #Reemplazar el compuesto en la formula
        example_compuesto = example_compuesto.replace(match.group(0), compuesto)
    if example_compuesto.count('{') > 0:
        match = re.search(r'\{(.*?)\}(\d+\.?\d*)', example_compuesto)
        compuesto = match.group(1)
        multiplicador = match.group(2)

        compuesto = re.sub(r'(\d+\.?\d*)', lambda x: str(float(x.group(1))* float(multiplicador)), compuesto)

        #Reemplazar el compuesto en la formula
        example_compuesto = example_compuesto.replace(match.group(0), compuesto)
    
    #Separar los numeros de la formula y guardarlos en una lista
    number = re.findall(r'\d+\.?\d*', example_compuesto)
    #Guardar en una lista lo restante de la formula
    element = re.split(r'\d+\.?\d*', example_compuesto)
    #Unir las listas en un diccionario
    dict_compuesto = dict(zip(element, number))
    #Convertir el diccionario en un dataframe
    return dict_compuesto

In [392]:
def addElementsAndCompositionPercentage(df):
    for index, row in df.iterrows():
        formula = row['Formula']
        #Aplicar la funcion para encontrar el porcentaje de composicion correcto de cada elemento
        composicion = findCompositionPercentage(formula)
        for i in range(len(composicion)):
            #Agregar la key a la columna Elem{i} y el valor a la columna Compo{i}
            df.loc[index, f'Elem{i+1}'] = list(composicion.keys())[i]
            df.loc[index, f'Compo{i+1}'] = list(composicion.values())[i]
            
    #Reemplazar los valores NaN por 0
    df.fillna(0, inplace=True)
    return df

In [391]:
#Validar composicion
def validateComposition(df):
    df_copy = df.copy()
    #Sumar los valores de las columnas Compo1, Compo2, etc
    for index, row in df_copy.iterrows():
        suma = 0
        for i in range(1,9):
            if row[f'Compo{i}'] != None:
                suma += float(row[f'Compo{i}'])
        df_copy.loc[index, 'Suma'] = suma

    #Intervalo de error permitido
    error = 1
    #Validar que la suma este dentro del intervalo de error
    df_copy['Validacion'] = df_copy['Suma'].apply(lambda x: True if x >= 100-error and x <= 100+error else False)
    return df_copy

In [399]:
def generateReport(df):
    #Eliminar la columna Validacion y Suma
    df_report = df.drop(['Validacion', 'Suma'], axis=1)
    #Guardar el reporte en un archivo csv
    df_report.to_csv("../Outputs/Report.csv", index=False)
    return df_report

In [389]:
def cleanDataset(dfGFA,dfElements):
    #Procesa las formulas que no tienen valores de proporcion
    dfNotNumbe = formulaWithoutNumber(dfGFA) #FUNCIONA
    #Unir los dfs
    dfGFA = (pd.concat([dfGFA, dfNotNumbe]))
    #Concatear al dfGFA el dfElements, las columnas 'Formula', 'Elem1','Elem2','Elem3','Elem4','Elem5','Elem6','Elem7','Elem8','CantElemen', Compo1, Compo2, Compo3, Compo4, Compo5, Compo6, Compo7, Compo8
    dfElementsOfCompos = pd.concat([dfGFA, pd.DataFrame(columns=['Elem1','Elem2','Elem3','Elem4','Elem5','Elem6','Elem7','Elem8','CantElemen','Compo1','Compo2','Compo3','Compo4','Compo5','Compo6','Compo7','Compo8'])])    
    #Aplicar la función addCompositionPercentage 
    dfGFA= addElementsAndCompositionPercentage (dfElementsOfCompos)
    dfGFA = validateComposition(dfGFA)
    dfGFA = generateReport(dfGFA)
    return dfGFA

In [400]:
dfGFA = loadGFA()
dfGFA = dfGFA
dfElements = loadTablaPeriodica()
df_tests = cleanDataset(dfGFA,dfElements)
df_tests


,Formula,Phase,Elem1,Elem2,Elem3,Elem4,Elem5,Elem6,Elem7,Elem8,CantElemen,Compo1,Compo2,Compo3,Compo4,Compo5,Compo6,Compo7,Compo8
0,Ag27Cu73,CRA,Ag,Cu,0,0,0,0,0,0,0,27,73,0,0,0,0,0,0
1,Ag35Ca65,BMG,Ag,Ca,0,0,0,0,0,0,0,35,65,0,0,0,0,0,0
2,Ag42Fe58,CRA,Ag,Fe,0,0,0,0,0,0,0,42,58,0,0,0,0,0,0
3,Ag48Cu52,CRA,Ag,Cu,0,0,0,0,0,0,0,48,52,0,0,0,0,0,0
4,Ag48Fe52,CRA,Ag,Fe,0,0,0,0,0,0,0,48,52,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,Zr20.0Hf20.0Ti20.0Cu20.0Fe20.0,BMG,Zr,Hf,Ti,Cu,Fe,0,0,0,0,20.0,20.0,20.0,20.0,20.0,0,0,0
6411,Zr20.0Hf20.0Ti20.0Cu20.0Ni20.0,BMG,Zr,Hf,Ti,Cu,Ni,0,0,0,0,20.0,20.0,20.0,20.0,20.0,0,0,0
6412,Al16.7Cr16.7Mo16.7Ta16.7Ti16.7Zr16.7,BMG,Al,Cr,Mo,Ta,Ti,Zr,0,0,0,16.7,16.7,16.7,16.7,16.7,16.7,0,0
6413,Sr16.7Ca16.7Yb16.7Mg16.7Zn16.7Cu16.7,BMG,Sr,Ca,Yb,Mg,Zn,Cu,0,0,0,16.7,16.7,16.7,16.7,16.7,16.7,0,0
